In [1]:
# loading dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
model_name = 'qanastek/51-languages-classifier'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

dataset = load_dataset("cakiki/rosetta-code",split='train')

In [33]:
print(dataset)
dataset[0]

Dataset({
    features: ['task_url', 'task_name', 'task_description', 'language_url', 'language_name', 'code'],
    num_rows: 79013
})


{'task_url': 'http://rosettacode.org/wiki/Ascending_primes',
 'task_name': 'Ascending primes',
 'task_description': "Generate and show all primes with strictly ascending decimal digits.\n\nAside: Try solving without peeking at existing solutions. I had a weird idea for generating\na prime sieve faster, which needless to say didn't pan out. The solution may be p(r)etty trivial\nbut generating them quickly is at least mildly interesting.\nTip: filtering all 7,027,260 primes below 123,456,789 probably won't kill you, but there is\nat least one significantly better and much faster way, needing a mere 511 odd/prime tests.\n\n\n\nSee also\n OEIS:A052015 - Primes with distinct digits in ascending order\n\n\nRelated\n\nPrimes with digits in nondecreasing order (infinite series allowing duplicate digits, whereas this isn't and doesn't)\nPandigital prime (whereas this is the smallest, with gaps in the used digits being permitted)\n\n",
 'language_url': '#ALGOL_68',
 'language_name': 'ALGOL 68',


In [54]:
# Extract the 'code' and 'language_name' columns
filtered_dataset = dataset.select_columns(['code','language_name'])

In [55]:
# filtered_dataset = filtered_dataset.select([0])

In [67]:
from datasets import load_dataset
import datasets

# Function to split code snippets into chunks and associate them with the language name
def split_and_associate_language(example):
    max_token_limit = 510
    code = example["code"]
    language_name = example["language_name"]  
    # Tokenize the code
    tokens = tokenizer.encode(code, add_special_tokens=True)
    # Split into chunks
    chunks = [tokens[i:i + max_token_limit] for i in range(0, len(tokens), max_token_limit)]
    # print(len(chunks))

    # Create a list of dictionaries, each representing a chunk
    # chunked_examples = [{"code": tokenizer.decode(chunk, skip_special_tokens=True), "language_name": language_name} for chunk in chunks]
    chunked_examples = {'code':[],'language_name':[]}
    # code = []
    # language = []
    for chunk in chunks:
        chunked_examples['code'].append(tokenizer.decode(chunk, skip_special_tokens=True))
        chunked_examples['language_name'].append(language_name)
        # code += [tokenizer.decode(chunk, skip_special_tokens=True)]
        # language += [language_name]
    # print(chunked_examples)
    # print(language)
    

    return chunked_examples#{'code':code,'language_name':language}#chunked_examples


filtered_dataset = filtered_dataset.map(split_and_associate_language)

flattened_dataset = {
    'code': [item for sublist in filtered_dataset['code'] for item in sublist],
    'language_name': [item for sublist in filtered_dataset['language_name'] for item in sublist],
}
new_dataset = datasets.Dataset.from_dict({'code': flattened_dataset['code'], 'language_name': flattened_dataset['language_name']})
new_dataset

Map:   0%|          | 0/79013 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

## Removing rows from dataset which has value less then certain threshold


In [100]:
new_dataset = datasets.load_from_disk('preprocessed_programming_language_data')
new_dataset

Dataset({
    features: ['code', 'language_name'],
    num_rows: 100816
})

In [101]:
label_counts
# new_dataset['language_name']

Counter({'REXX': 1867,
         'Go': 1806,
         'C': 1728,
         'Phix': 1695,
         'C++': 1665,
         'Wren': 1632,
         'Java': 1626,
         'Nim': 1594,
         'Python': 1524,
         'Julia': 1457,
         'Raku': 1420,
         'Perl': 1416,
         'Kotlin': 1379,
         'Racket': 1299,
         'Haskell': 1282,
         'D': 1251,
         'C#': 1238,
         'Rust': 1235,
         'Ada': 1148,
         'Ruby': 1126,
         'Fortran': 1121,
         'J': 1118,
         'FreeBASIC': 1114,
         'Tcl': 1082,
         'JavaScript': 1065,
         'Scala': 1051,
         'ALGOL 68': 1029,
         'zkl': 991,
         'ARM Assembly': 960,
         'Lua': 959,
         'AutoHotkey': 957,
         '11l': 929,
         'Common Lisp': 919,
         'Sidef': 910,
         'Pascal': 883,
         'Factor': 877,
         'PicoLisp': 830,
         'AArch64 Assembly': 826,
         'Delphi': 807,
         'F#': 776,
         'PureBasic': 775,
         'jq': 

In [104]:
from collections import Counter
label_counts = Counter(new_dataset['language_name'])
threshold = 500
labels_to_keep = []
for label,count in label_counts.items():
    if count> threshold:
        # print(label,count)
        labels_to_keep.append(label)
# def filter_labels(examples):
#     return {'language_name': [label for label in examples['language_name'] if label in labels_to_keep]}

new_dataset = new_dataset.filter(lambda example: example['language_name'] in labels_to_keep)
new_dataset

Filter:   0%|          | 0/100816 [00:00<?, ? examples/s]

Dataset({
    features: ['code', 'language_name'],
    num_rows: 66148
})

In [105]:
label_counts = Counter(new_dataset_less['language_name'])
label_counts


Counter({'REXX': 1867,
         'Go': 1806,
         'C': 1728,
         'Phix': 1695,
         'C++': 1665,
         'Wren': 1632,
         'Java': 1626,
         'Nim': 1594,
         'Python': 1524,
         'Julia': 1457,
         'Raku': 1420,
         'Perl': 1416,
         'Kotlin': 1379,
         'Racket': 1299,
         'Haskell': 1282,
         'D': 1251,
         'C#': 1238,
         'Rust': 1235,
         'Ada': 1148,
         'Ruby': 1126,
         'Fortran': 1121,
         'J': 1118,
         'FreeBASIC': 1114,
         'Tcl': 1082,
         'JavaScript': 1065,
         'Scala': 1051,
         'ALGOL 68': 1029,
         'zkl': 991,
         'ARM Assembly': 960,
         'Lua': 959,
         'AutoHotkey': 957,
         '11l': 929,
         'Common Lisp': 919,
         'Sidef': 910,
         'Pascal': 883,
         'Factor': 877,
         'PicoLisp': 830,
         'AArch64 Assembly': 826,
         'Delphi': 807,
         'F#': 776,
         'PureBasic': 775,
         'jq': 

In [107]:
new_dataset.save_to_disk('preprocessed_programming_language_data_count_great500')
new_dataset.to_csv('preprocessed_programming_language_data_count_great500.csv')

Saving the dataset (0/1 shards):   0%|          | 0/66148 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/67 [00:00<?, ?ba/s]

44344014

In [68]:
label_counts = Counter(new_dataset['language_name'])
label_counts

Counter({'REXX': 1867,
         'C': 1728,
         'C++': 1665,
         'Java': 1626,
         'Julia': 1457,
         'Raku': 1420,
         'Racket': 1299,
         'D': 1251,
         'C#': 1238,
         'Rust': 1235,
         'Ruby': 1126,
         'J': 1118,
         'JavaScript': 1065,
         'Common Lisp': 919,
         'Delphi': 807,
         'Ring': 748,
         'Clojure': 687,
         'R': 598,
         'COBOL': 523,
         'Run BASIC': 303,
         'Crystal': 256,
         'CLU': 244,
         'CoffeeScript': 234,
         'Red': 171,
         'Jsish': 154,
         'REBOL': 148,
         'Cowgol': 148,
         'Dyalect': 138,
         'RATFOR': 132,
         'Ceylon': 121,
         'Dart': 116,
         'Retro': 103,
         'Déjà Vu': 103,
         'DWScript': 101,
         'Component Pascal': 88,
         'Draco': 75,
         'Raven': 73,
         'Commodore BASIC': 69,
         'ColdFusion': 66,
         'Caché ObjectScript': 54,
         'Rascal': 53,
     

In [15]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.Series(new_dataset['language_name']).value_counts()

#### Data Observation

In [65]:
len(set(new_dataset['language_name']))

154

In [8]:
data = pd.DataFrame(list(zip(dataset['code'],dataset['language_name'])),columns = ['code','language_name'],index = None)
# pd.DataFrame(list(zip(lst, lst2)),
#                columns =['Name', 'val'])
data['code'] = data['code'].apply(lambda x: x.replace('\n',' '))
data

,code,language_name
0,BEGIN # find all primes with strictly increasi...,ALGOL 68
1,ascending?: function [x][ initial: digits ...,Arturo
2,# syntax: GAWK -f ASCENDING_PRIMES.AWK BEGIN...,AWK
3,// Ascending primes. Nigel Galloway: April 1...,F#
4,USING: grouping math math.combinatorics math.f...,Factor
...,...,...
79008,"do(2){ask(""A B: "").split("" "").filter().sum().p...",zkl
79009,program: a_plus_b input: '7 11' output: 1...,Zoea
79010,"module ABProblem; var \ta,b: integer; begin ...",Zoea Visual
79011,"module ABProblem; var \ta,b: integer; begin ...",zonnon


In [66]:
data = pd.DataFrame(list(zip(new_dataset['code'],new_dataset['language_name'])),columns = ['code','language_name'],index = None)
# pd.DataFrame(list(zip(lst, lst2)),
#                columns =['Name', 'val'])
data['code'] = data['code'].apply(lambda x: x.replace('\n',' '))
data

,code,language_name
0,"extend=: {{ y;(1+each i._1+{.y),L:0 y }} $(#~ ...",J
1,using Combinatorics using Primes function asce...,Julia
2,"put (flat 2, 3, 5, 7, sort +*, gather (1..8).m...",Raku
3,"load ""stdlibcore.ring"" limit = 1000 row = 0 fo...",Ring
4,#include <stdlib.h> #include <stdio.h> #includ...,C
...,...,...
26242,"input string ""Input A:"" set ""A"" to ""input"" inp...",Robotic
26243,Listen to A number Listen to B Say A number pl...,Rockstar
26244,puts gets.split.sum(&:to_i),Ruby
26245,"input, x$ print val(word$(x$,1)) + val(word$(x...",Run BASIC
